In [1]:
%%time 

# CPU times: user 4.06 ms, sys: 418 µs, total: 4.48 ms
# Wall time: 10.7 s

SAVEPATH = "s3a://team-cedarsdata-582958291898-mvvpu/"
# FILE = "AIS_sample_baltic_005_202201_202206.parquet"
# FILE = "AIS_sample_baltic_025_202201_202206.parquet"
# FILE = "AIS_sample_baltic_005_202201_202206a.parquet"
FILE = "AIS_sample_baltic_0025_202101_202106.parquet"
ais = spark.read.parquet(SAVEPATH+FILE)
ais.show()

+---------+------------------+------------------+--------------------+-----------+------------------+--------------------+----------------+--------------------+------------+-------------------+-------------------+------+-----+----------------+-------------------+--------+------+------------------+------------------+
|     mmsi|          latitude|         longitude|         vessel_name|vessel_type|  vessel_type_main|     vessel_type_sub|vessel_type_code|   vessel_type_cargo|vessel_class|      dt_insert_utc|               eeid|length|width|    flag_country|        destination|     eta|source|    H3_int_index_1|    H3_int_index_5|
+---------+------------------+------------------+--------------------+-----------+------------------+--------------------+----------------+--------------------+------------+-------------------+-------------------+------+-----+----------------+-------------------+--------+------+------------------+------------------+
|272102200| 46.94711333333333|31.9923016666666

In [2]:
%%time

# CPU times: user 2.27 ms, sys: 0 ns, total: 2.27 ms
# Wall time: 13.7 s

ais.count()

CPU times: user 2.27 ms, sys: 0 ns, total: 2.27 ms
Wall time: 13.7 s


9523854

In [2]:
from datetime import datetime, timedelta

from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import unix_timestamp, from_unixtime, to_date, lit, when, col, ntile, row_number, lag, lead, count, countDistinct
from pyspark.sql.functions import array, array_union, explode
from pyspark.sql.functions import year, month, dayofmonth, substring
from pyspark.sql.functions import col, radians, asin, sin, sqrt, cos, acos
import pyspark.sql.functions as f
from pyspark.sql.functions import broadcast
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType, DoubleType

import pandas as pd
import numpy as np
from shapely.geometry import Point

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)

import warnings

In [3]:
spark

In [4]:
spark.conf.set("spark.sql.broadcastTimeout", "50000")
spark

In [5]:
%%time

# CPU times: user 170 ms, sys: 7.87 ms, total: 178 ms
# Wall time: 621 ms

url = 'https://raw.githubusercontent.com/datacerdas/know-ais/main/platform/src/data/ports_.csv'
ports_df = pd.read_csv(url, error_bad_lines=False)

ports = spark.createDataFrame(ports_df) 
ports.show()

+----------+--------------------+---------+--------+--------+----+-----+------------------+
|Unnamed: 0|                name|  port_id|lat_port|lon_port|iso2| area|          h3_5_int|
+----------+--------------------+---------+--------+--------+----+-----+------------------+
|         0|            Aamshite|wfp.16923|34.15214|35.62828|  lb|black|599781562506018815|
|         0|            Aamshite|wfp.16923|34.15214|35.62828|  lb|black|599781563579760639|
|         1|            Aamshite|wfp.16923|34.15214|35.62828|  lb|black|599781564653502463|
|         2|            Aamshite|wfp.16923|34.15214|35.62828|  lb|black|599781534588731391|
|         3|            Aamshite|wfp.16923|34.15214|35.62828|  lb|black|599781535662473215|
|         4|            Aamshite|wfp.16923|34.15214|35.62828|  lb|black|599781550694858751|
|         5|            Aamshite|wfp.16923|34.15214|35.62828|  lb|black|599781558211051519|
|         1|Aamshite Fuel Oil...|wfp.17891|34.15434|35.63044|  lb|black|59978156

In [6]:
%%time

# CPU times: user 12.7 ms, sys: 4.16 ms, total: 16.9 ms
# Wall time: 25.7 s

ais_00 = ais\
        .select(
                'mmsi','eeid',col('dt_insert_utc').alias('timestamp'),col('longitude').alias('lon'),col('latitude').alias('lat'),
                'vessel_name','vessel_type','vessel_type_main','vessel_type_sub','vessel_type_code','vessel_type_cargo','vessel_class',
                'length','width','flag_country','destination','eta','source','H3_int_index_5'
            )\
        .sort('mmsi', 'timestamp')

ais_00.show(10)

+----+-------------------+-------------------+-----------+-----------+------------------+--------------+----------------+--------------------+----------------+--------------------+------------+------+-----+------------------+--------------------+-------+------+------------------+
|mmsi|               eeid|          timestamp|        lon|        lat|       vessel_name|   vessel_type|vessel_type_main|     vessel_type_sub|vessel_type_code|   vessel_type_cargo|vessel_class|length|width|      flag_country|         destination|    eta|source|    H3_int_index_5|
+----+-------------------+-------------------+-----------+-----------+------------------+--------------+----------------+--------------------+----------------+--------------------+------------+------+-----+------------------+--------------------+-------+------+------------------+
|null|5187421282254041088|2021-06-17 07:33:13|   11.05548|   55.47962|            HJARNO|         Cargo|            null|                null|              7

In [7]:
ais_00.cache()

DataFrame[mmsi: int, eeid: bigint, timestamp: timestamp, lon: double, lat: double, vessel_name: string, vessel_type: string, vessel_type_main: string, vessel_type_sub: string, vessel_type_code: int, vessel_type_cargo: string, vessel_class: string, length: double, width: double, flag_country: string, destination: string, eta: int, source: string, H3_int_index_5: bigint]

In [8]:
%%time

# CPU times: user 11 ms, sys: 0 ns, total: 11 ms
# Wall time: 1min 22s

ais_01_ports = ais_00\
                    .join(
                        ports\
                          .withColumn('H3_int_index_5', col('h3_5_int'))
                        , ['H3_int_index_5'], 'left'
                    )

ais_01_ports.show(10)

+------------------+----+-------------------+-------------------+-----------+-----------+----------------+--------------+------------------+---------------+----------------+--------------------+------------+------+-----+-------------+------------------+--------+------+----------+---------+-------+--------+--------+----+------+------------------+
|    H3_int_index_5|mmsi|               eeid|          timestamp|        lon|        lat|     vessel_name|   vessel_type|  vessel_type_main|vessel_type_sub|vessel_type_code|   vessel_type_cargo|vessel_class|length|width| flag_country|       destination|     eta|source|Unnamed: 0|     name|port_id|lat_port|lon_port|iso2|  area|          h3_5_int|
+------------------+----+-------------------+-------------------+-----------+-----------+----------------+--------------+------------------+---------------+----------------+--------------------+------------+------+-----+-------------+------------------+--------+------+----------+---------+-------+------

In [9]:
ais_01_ports.cache()

DataFrame[H3_int_index_5: bigint, mmsi: int, eeid: bigint, timestamp: timestamp, lon: double, lat: double, vessel_name: string, vessel_type: string, vessel_type_main: string, vessel_type_sub: string, vessel_type_code: int, vessel_type_cargo: string, vessel_class: string, length: double, width: double, flag_country: string, destination: string, eta: int, source: string, Unnamed: 0: bigint, name: string, port_id: string, lat_port: double, lon_port: double, iso2: string, area: string, h3_5_int: bigint]

In [ ]:
%%time 

# CPU times: user 28.7 ms, sys: 1.37 ms, total: 30.1 ms
# Wall time: 27.5 s

ais_01_ports.filter(col('port_id').isNotNull()).count()

In [10]:
%%time

# CPU times: user 42 ms, sys: 15.5 ms, total: 57.5 ms
# Wall time: 29.2 s

part = Window.partitionBy('mmsi').orderBy(['timestamp'])
part2 = Window.partitionBy('mmsi').orderBy(['timestamp']).rowsBetween(Window.unboundedPreceding, Window.currentRow)

ais_02_movement = ais_01_ports\
                    .dropDuplicates(['mmsi','timestamp','lat','lon'])\
                    .withColumn('in_port', when(col('port_id').isNotNull(), 1).otherwise(0))\
                    .withColumn('lag_in_port', f.lag('in_port').over(part) )\
                    .withColumn('in_port_gap', when( col('lag_in_port') != col('in_port'), 1 ).otherwise(0) )\
                    .withColumn('lag_port_id', f.lag('port_id').over(part) )\
                    .withColumn('port_gap', when( col('lag_port_id') != col('port_id'), 1 ).otherwise(0) )\
                    .withColumn('stop_gap', when( ( col('in_port_gap') == 1 ) | ( col('port_gap') == 1 ), 1 ).otherwise(0) )\
                    .withColumn('stop_movement', f.sum('stop_gap').over(part2))

ais_02_movement.show(10)

+------------------+----+----+-------------------+------------------+------------------+-----------------+-------------+----------------+---------------+----------------+-----------------+------------+------+-----+------------+-----------+----+------+----------+---------+-------+--------+--------+----+------+------------------+-------+-----------+-----------+-----------+--------+--------+-------------+
|    H3_int_index_5|mmsi|eeid|          timestamp|               lon|               lat|      vessel_name|  vessel_type|vessel_type_main|vessel_type_sub|vessel_type_code|vessel_type_cargo|vessel_class|length|width|flag_country|destination| eta|source|Unnamed: 0|     name|port_id|lat_port|lon_port|iso2|  area|          h3_5_int|in_port|lag_in_port|in_port_gap|lag_port_id|port_gap|stop_gap|stop_movement|
+------------------+----+----+-------------------+------------------+------------------+-----------------+-------------+----------------+---------------+----------------+-----------------+

In [11]:
ais_02_movement.cache()

DataFrame[H3_int_index_5: bigint, mmsi: int, eeid: bigint, timestamp: timestamp, lon: double, lat: double, vessel_name: string, vessel_type: string, vessel_type_main: string, vessel_type_sub: string, vessel_type_code: int, vessel_type_cargo: string, vessel_class: string, length: double, width: double, flag_country: string, destination: string, eta: int, source: string, Unnamed: 0: bigint, name: string, port_id: string, lat_port: double, lon_port: double, iso2: string, area: string, h3_5_int: bigint, in_port: int, lag_in_port: int, in_port_gap: int, lag_port_id: string, port_gap: int, stop_gap: int, stop_movement: bigint]

In [12]:
%%time

# CPU times: user 61.6 ms, sys: 32.1 ms, total: 93.7 ms
# Wall time: 1min 19s

part3 = Window.partitionBy(['mmsi','stop_movement']).orderBy(['timestamp'])
part4 = Window.partitionBy(['mmsi','stop_movement'])

ais_03_movement = ais_02_movement\
                    .withColumn('lag_lat', f.lag('lat').over(part3))\
                    .withColumn('lag_lon', f.lag('lon').over(part3))\
                    .withColumn('lag_timestamp', f.lag('timestamp').over(part3))\
                    .withColumn('time_interval', f.to_timestamp(col('timestamp')).cast('double') - f.to_timestamp(col('lag_timestamp')).cast('double')  )\
                    .withColumn('time_interval_hour', f.round(col('time_interval')/3600, 2) )\
                    .withColumn('distance',  \
                                  f.round( \
                                    6371.0 * f.acos( \
                                        f.cos(f.radians('lat')) * f.cos(f.radians('lag_lat')) \
                                        * f.cos(f.radians('lon') - f.radians('lag_lon')) \
                                        + f.sin(f.radians('lag_lat')) * f.sin(f.radians('lat'))) \
                                  , 2 ) \
                                )\
                    .withColumn('speed_kmh', f.round( col('distance') / (col('time_interval') / 3600) ,2) )\
                    .withColumn('start_timestamp', f.min('timestamp').over(part4) )\
                    .withColumn('end_timestamp', f.max('timestamp').over(part4) )\
                    .withColumn('min_speed', f.min('speed_kmh').over(part4) )\
                    .withColumn('avg_speed', f.avg('speed_kmh').over(part4) )\
                    .withColumn('max_speed', f.max('speed_kmh').over(part4) )\
                    .withColumn('sum_distance', f.sum('distance').over(part4) )\
                    .withColumn('movement_coordinates', f.collect_list(f.array('lon', 'lat')).over(part4) )

ais_03_movement.show(10)

+------------------+----+----+-------------------+------------------+------------------+-----------------+-------------+----------------+---------------+----------------+-----------------+------------+------+-----+------------+-----------+----+------+----------+---------+-------+--------+--------+----+------+------------------+-------+-----------+-----------+-----------+--------+--------+-------------+------------------+------------------+-------------------+-------------+------------------+--------+---------+-------------------+-------------------+---------+--------------------+---------+-----------------+--------------------+
|    H3_int_index_5|mmsi|eeid|          timestamp|               lon|               lat|      vessel_name|  vessel_type|vessel_type_main|vessel_type_sub|vessel_type_code|vessel_type_cargo|vessel_class|length|width|flag_country|destination| eta|source|Unnamed: 0|     name|port_id|lat_port|lon_port|iso2|  area|          h3_5_int|in_port|lag_in_port|in_port_gap|lag

In [13]:
ais_03_movement.cache()

DataFrame[H3_int_index_5: bigint, mmsi: int, eeid: bigint, timestamp: timestamp, lon: double, lat: double, vessel_name: string, vessel_type: string, vessel_type_main: string, vessel_type_sub: string, vessel_type_code: int, vessel_type_cargo: string, vessel_class: string, length: double, width: double, flag_country: string, destination: string, eta: int, source: string, Unnamed: 0: bigint, name: string, port_id: string, lat_port: double, lon_port: double, iso2: string, area: string, h3_5_int: bigint, in_port: int, lag_in_port: int, in_port_gap: int, lag_port_id: string, port_gap: int, stop_gap: int, stop_movement: bigint, lag_lat: double, lag_lon: double, lag_timestamp: timestamp, time_interval: double, time_interval_hour: double, distance: double, speed_kmh: double, start_timestamp: timestamp, end_timestamp: timestamp, min_speed: double, avg_speed: double, max_speed: double, sum_distance: double, movement_coordinates: array<array<double>>]

In [14]:
%%time

# CPU times: user 16.4 ms, sys: 2.35 ms, total: 18.8 ms
# Wall time: 6.36 s

ais_04_movement = ais_03_movement\
                    .select([
                        'mmsi',
                        'timestamp',
                        'lon',
                        'lat',
                        'vessel_name',
                        'vessel_type',
                        'vessel_type_main',
                        'vessel_type_sub',
                        'vessel_type_code',
                        'vessel_type_cargo',
                        'vessel_class',
                        'length',
                        'width',
                        'flag_country',
                        'destination',
                        'eta',
                        'source',
                        'name',
                        'port_id',
                        'lat_port',
                        'lon_port',
                        'iso2',
                        'H3_int_index_5',
                        'time_interval',
                        'time_interval_hour',
                        'speed_kmh',
                        'start_timestamp',
                        'end_timestamp',
                        'min_speed',
                        'avg_speed',
                        'max_speed',
                        col('sum_distance').alias('distance'),
                        'movement_coordinates'
                    ])\
                    .dropDuplicates(['mmsi','start_timestamp', 'end_timestamp'])
  
ais_04_movement.show(10)

+---------+-------------------+------------------+------------------+-----------------+-------------+----------------+---------------+----------------+-----------------+------------+------+-----+------------+-----------+-------+------+----------+---------+--------+--------+----+------------------+-------------+------------------+---------+-------------------+-------------------+---------+--------------------+---------+-----------------+--------------------+
|     mmsi|          timestamp|               lon|               lat|      vessel_name|  vessel_type|vessel_type_main|vessel_type_sub|vessel_type_code|vessel_type_cargo|vessel_class|length|width|flag_country|destination|    eta|source|      name|  port_id|lat_port|lon_port|iso2|    H3_int_index_5|time_interval|time_interval_hour|speed_kmh|    start_timestamp|      end_timestamp|min_speed|           avg_speed|max_speed|         distance|movement_coordinates|
+---------+-------------------+------------------+------------------+-------

In [15]:
ais_04_movement.cache()

DataFrame[mmsi: int, timestamp: timestamp, lon: double, lat: double, vessel_name: string, vessel_type: string, vessel_type_main: string, vessel_type_sub: string, vessel_type_code: int, vessel_type_cargo: string, vessel_class: string, length: double, width: double, flag_country: string, destination: string, eta: int, source: string, name: string, port_id: string, lat_port: double, lon_port: double, iso2: string, H3_int_index_5: bigint, time_interval: double, time_interval_hour: double, speed_kmh: double, start_timestamp: timestamp, end_timestamp: timestamp, min_speed: double, avg_speed: double, max_speed: double, distance: double, movement_coordinates: array<array<double>>]

In [16]:
%%time

# CPU times: user 20.3 ms, sys: 4.21 ms, total: 24.5 ms
# Wall time: 2.51 s

part5 = Window.partitionBy('mmsi').orderBy(['start_timestamp'])
part6 = Window.partitionBy('mmsi').orderBy(['start_timestamp']).rowsBetween(Window.unboundedPreceding, Window.currentRow)

ais_05_shipping = ais_04_movement\
                    .withColumn('lag_start_timestamp', f.lag('start_timestamp').over(part5) )\
                    .withColumn('lag_end_timestamp', f.lag('end_timestamp').over(part5) )\
                    .withColumn('lead_start_timestamp', f.lead('start_timestamp').over(part5) )\
                    .withColumn('lead_end_timestamp', f.lead('end_timestamp').over(part5) )\
                    .withColumn('lag_time_interval', f.to_timestamp(col('start_timestamp')).cast('double') - f.to_timestamp(col('lag_end_timestamp')).cast('double')  )\
                    .withColumn('lead_time_interval', f.to_timestamp(col('lead_start_timestamp')).cast('double') - f.to_timestamp(col('end_timestamp')).cast('double')  )\
                    .withColumn('shipping_status', when( col('lag_time_interval').isNull(), 0 )\
                                                    .when( col('lag_time_interval') > 10000, 0 )\
                                                    .when( col('lead_time_interval').isNull(), 2 )\
                                                    .when( col('lead_time_interval') > 10000, 2 ).otherwise(1) )\
                    .withColumn('shipping_gap', when( col('shipping_status') == 0, 1).otherwise(0) )\
                    .withColumn('shipping_movement', f.sum('shipping_gap').over(part6) )

ais_05_shipping.show(10)

+---------+-------------------+------------------+------------------+-----------------+-------------+----------------+---------------+----------------+-----------------+------------+------+-----+------------+-----------+-------+------+----------+---------+--------+--------+----+------------------+-------------+------------------+---------+-------------------+-------------------+---------+--------------------+---------+-----------------+--------------------+-------------------+-------------------+--------------------+-------------------+-----------------+------------------+---------------+------------+-----------------+
|     mmsi|          timestamp|               lon|               lat|      vessel_name|  vessel_type|vessel_type_main|vessel_type_sub|vessel_type_code|vessel_type_cargo|vessel_class|length|width|flag_country|destination|    eta|source|      name|  port_id|lat_port|lon_port|iso2|    H3_int_index_5|time_interval|time_interval_hour|speed_kmh|    start_timestamp|      end_ti

In [18]:
ais_05_shipping.cache()

DataFrame[mmsi: int, timestamp: timestamp, lon: double, lat: double, vessel_name: string, vessel_type: string, vessel_type_main: string, vessel_type_sub: string, vessel_type_code: int, vessel_type_cargo: string, vessel_class: string, length: double, width: double, flag_country: string, destination: string, eta: int, source: string, name: string, port_id: string, lat_port: double, lon_port: double, iso2: string, H3_int_index_5: bigint, time_interval: double, time_interval_hour: double, speed_kmh: double, start_timestamp: timestamp, end_timestamp: timestamp, min_speed: double, avg_speed: double, max_speed: double, distance: double, movement_coordinates: array<array<double>>, lag_start_timestamp: timestamp, lag_end_timestamp: timestamp, lead_start_timestamp: timestamp, lead_end_timestamp: timestamp, lag_time_interval: double, lead_time_interval: double, shipping_status: int, shipping_gap: int, shipping_movement: bigint]

In [ ]:
# write as parquet to s3
# ais_05_shipping.repartition(1).write.mode('overwrite').parquet(SAVEPATH+"RESULT_ais_05_shipping_005_202201_202206a.parquet")

In [ ]:
%%time

# CPU times: user 47.9 ms, sys: 15.1 ms, total: 62.9 ms
# Wall time: 2min 40s

part7 = Window.partitionBy('id_shipping')

ais_06_shipping = ais_05_shipping\
                    .withColumn('id_shipping', f.concat('mmsi','shipping_movement'))\
                    .withColumn('start_time', f.first('start_timestamp').over(part7))\
                    .withColumn('end_time', f.last('end_timestamp').over(part7))\
                    .withColumn('origin_port_id', f.first('port_id',ignorenulls=True).over(part7) )\
                    .withColumn('origin_port_name', f.first('name',ignorenulls=True).over(part7) )\
                    .withColumn('origin_country_iso2', f.first('iso2',ignorenulls=True).over(part7) )\
                    .withColumn('destination_port_id', f.last('port_id',ignorenulls=True).over(part7) )\
                    .withColumn('destination_port_name', f.last('name',ignorenulls=True).over(part7) )\
                    .withColumn('destination_country_iso2', f.last('iso2',ignorenulls=True).over(part7) )

ais_06_shipping.show(10)

+---------+-------------------+------------------+------------------+-------------+--------------+----------------+---------------+----------------+-----------------+------------+------+-----+------------+-----------------+-------+------+-----------------+---------+--------+--------+----+------------------+-------------+------------------+---------+-------------------+-------------------+---------+-------------------+---------+--------+--------------------+-------------------+-------------------+--------------------+-------------------+-----------------+------------------+---------------+------------+-----------------+-----------+-------------------+-------------------+--------------+-----------------+-------------------+-------------------+---------------------+------------------------+
|     mmsi|          timestamp|               lon|               lat|  vessel_name|   vessel_type|vessel_type_main|vessel_type_sub|vessel_type_code|vessel_type_cargo|vessel_class|length|width|flag_coun

In [20]:

part7 = Window.partitionBy('id_shipping')

ais_06_shipping = ais_06_shipping\
                    .withColumn('avg_speed', f.avg('avg_speed').over(part7) )\
                    .withColumn('sum_distance', f.sum('distance').over(part7) )\
                    .withColumn('is_imputed_port', when( f.first('port_id').over(part7).isNull(), 1 ).when( f.last('port_id').over(part7).isNull(), 1 ).otherwise(0) )\
                    .withColumn('movements', f.flatten(f.collect_list('movement_coordinates').over(part7)) )\
                    .select([
                        'id_shipping', 'mmsi', 
                        'vessel_name',
                        'vessel_type',
                        'vessel_type_main',
                        'vessel_type_sub',
                        'vessel_type_code',
                        'vessel_type_cargo',
                        'vessel_class',
                        'length',
                        'width',
                        'flag_country',
                        'destination',
                        'eta',
                        'source',
                        'start_time', 'end_time', 'movements',
                        'origin_port_id', 'origin_port_name', 'origin_country_iso2',
                        'destination_port_id', 'destination_port_name', 'destination_country_iso2',
                        'avg_speed', col('sum_distance').alias('distance'), 'is_imputed_port', 
                    ])\
                    .dropDuplicates(['id_shipping'])

ais_06_shipping.show(10)

+-----------+---------+-------------+--------------+----------------+---------------+----------------+-----------------+------------+------+-----+------------+-----------------+-------+------+-------------------+-------------------+--------------------+--------------+-----------------+-------------------+-------------------+---------------------+------------------------+-------------------+--------+---------------+
|id_shipping|     mmsi|  vessel_name|   vessel_type|vessel_type_main|vessel_type_sub|vessel_type_code|vessel_type_cargo|vessel_class|length|width|flag_country|      destination|    eta|source|         start_time|           end_time|           movements|origin_port_id| origin_port_name|origin_country_iso2|destination_port_id|destination_port_name|destination_country_iso2|          avg_speed|distance|is_imputed_port|
+-----------+---------+-------------+--------------+----------------+---------------+----------------+-----------------+------------+------+-----+------------+---

In [22]:
ais_06_shipping.cache()

DataFrame[id_shipping: string, mmsi: int, vessel_name: string, vessel_type: string, vessel_type_main: string, vessel_type_sub: string, vessel_type_code: int, vessel_type_cargo: string, vessel_class: string, length: double, width: double, flag_country: string, destination: string, eta: int, source: string, start_time: timestamp, end_time: timestamp, movements: array<array<double>>, origin_port_id: string, origin_port_name: string, origin_country_iso2: string, destination_port_id: string, destination_port_name: string, destination_country_iso2: string, avg_speed: double, distance: double, is_imputed_port: int]

In [23]:
ais_06_shipping.count()

459614

In [24]:
# write as parquet to s3
ais_06_shipping.repartition(1).write.mode('overwrite').parquet(SAVEPATH+"RESULT_AIS_sample_baltic_0025_202101_202106.parquet")

In [23]:
%%time 

# CPU times: user 4.06 ms, sys: 418 µs, total: 4.48 ms
# Wall time: 10.7 s

SAVEPATH = "s3a://team-cedarsdata-582958291898-mvvpu/"
FILE = "RESULT_AIS_sample_baltic_005_202201_202206.parquet"
result_ais = spark.read.parquet(SAVEPATH+FILE)
result_ais.show()

+-----------+---------+--------------------+--------------+------------------+---------------+----------------+--------------------+------------+------+-----+------------+---------------+--------+------+-------------------+-------------------+--------------------+--------------+-----------------+-------------------+-------------------+---------------------+------------------------+--------------------+------------------+---------------+
|id_shipping|     mmsi|         vessel_name|   vessel_type|  vessel_type_main|vessel_type_sub|vessel_type_code|   vessel_type_cargo|vessel_class|length|width|flag_country|    destination|     eta|source|         start_time|           end_time|           movements|origin_port_id| origin_port_name|origin_country_iso2|destination_port_id|destination_port_name|destination_country_iso2|           avg_speed|          distance|is_imputed_port|
+-----------+---------+--------------------+--------------+------------------+---------------+----------------+-------

In [27]:
result_ais.groupby(['origin_port_id']).agg(f.count('*').alias('top')).sort('top', ascending=False).show(10)

+--------------+------+
|origin_port_id|   top|
+--------------+------+
|          null|150918|
|     wfp.17151| 24300|
|     wfp.16140| 16978|
|           111| 12109|
|     wfp.14332|  9202|
|     wfp.15678|  7607|
|          4305|  7284|
|          2321|  7078|
|     wfp.14959|  6417|
|     wfp.17356|  6017|
+--------------+------+
only showing top 10 rows



In [31]:
result_ais.filter(col('origin_port_id') != col('destination_port_id')).count()

29371